File created by following https://marcotcr.github.io/lime/tutorials/Tutorial%20-%20continuous%20and%20categorical%20features.html

In [ ]:
from xaibenchmark import load_adult as la

In [ ]:
import sklearn
import sklearn.ensemble
import numpy as np
import lime
import lime.lime_tabular
import pandas as pd

## Load Dataset

In [ ]:
data = la.load_csv_data('adult', root_path='../data')

In [ ]:
data.data

In [ ]:
data.keys()

In [ ]:
set(data.data.workclass)

In [ ]:
data.categorical_features.keys()

In [ ]:
cats = pd.DataFrame.from_dict(data.categorical_features, orient='index').transpose()
ohe = sklearn.preprocessing.OneHotEncoder()
_ = ohe.fit_transform(cats)

In [ ]:
def preprocess(*data_df): 
    def process_single(df):
        
        cat_df = pd.get_dummies(df, columns=data.categorical_features.keys())
        missing_cols = {cat+'_'+str(attr) for cat in data.categorical_features for attr in data.categorical_features[cat]} - set(cat_df.columns)
        for c in missing_cols:
            cat_df[c] = 0
        return cat_df
        
    # Preprocess function for one-hot encoding categorical data
    #return [pd.get_dummies(df, columns=data.categorical_features.keys()) for df in data_df]
    return [process_single(df) for df in data_df]

In [ ]:
train, test = preprocess(data.data, data.data_test)
labels_train, labels_test = data.target, data.target_test

In [ ]:
train

## Train RF Classifier

In [ ]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(train, labels_train.to_numpy().reshape(-1))

In [ ]:
y_pred = rf.predict(train)

In [ ]:
y_true = labels_train.to_numpy().reshape(-1)

## Explain instances

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=train.keys(), class_names=data.target_names, discretize_continuous=False)

In [ ]:
i = np.random.randint(0, test.shape[0]) # get random sample

In [ ]:
exp = explainer.explain_instance(test.iloc[i], rf.predict_proba, num_features=10)

In [ ]:
data.data.iloc[i]

In [ ]:
print('True label:', data.target.iloc[i]['income'])
exp.show_in_notebook(show_table=False, show_all=False, show_predicted_value=True)
fig = exp.as_pyplot_figure()